In [17]:
# Importing packages
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import keras
import keras_tuner as kt
from keras.utils import to_categorical
from keras.layers import Dense, Convolution1D, Convolution2D, MaxPool1D, Flatten, Dropout, AveragePooling1D
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight
from sklearn.utils import resample
import warnings
import os
warnings.filterwarnings('ignore')

In [18]:
v_peaks_data = np.array(   [[1.0, 2.0, 1.5, 4.0, 9.0, 1.0, 2.3, 5.0, 1.0],
                            [1.0, 2.0, 1.5, 4.0, 9.0, 1.0, 0.0, 0.0, 0.0],
                            [1.0, 2.0, 1.5, 4.0, 9.0, 1.0, 2.3, 5.0, 1.0],
                            [1.0, 2.0, 1.5, 4.0, 9.0, 1.0, 2.3, 5.0, 0.0]])
v_peaks_time = np.array(   [[1, 2, 3, 4, 5, 6, 7, 8, 9],
                            [1, 2, 3, 4, 5, 6, 0, 0, 0],
                            [1, 2, 3, 4, 5, 6, 7, 8, 9],
                            [1, 2, 3, 4, 5, 6, 7, 8, 0]])

In [19]:
# Define structure of arraysù
NUM_PEAKS = 8
input_peaks = np.zeros((len(v_peaks_data),2*NUM_PEAKS))

# Fill strcuture with peaks data
# P0,dT0,P1,dT1,P2,dT2...,Pn,Tn  
# with Px: peak amplitude, Tx: delta time from previous peak
for n in range(len(v_peaks_data)):
    prev_time = 0
    tmp_data = v_peaks_data[n][v_peaks_time[n] != 0]
    tmp_time = v_peaks_time[n][v_peaks_time[n] != 0]
    # Calculate how many max and min peaks to consider
    if(len(tmp_data)>=NUM_PEAKS):
        n_max = int(NUM_PEAKS/2)
        n_min = NUM_PEAKS-n_max
    else:
        n_max = int(len(tmp_data)/2)
        n_min = len(tmp_data)-n_max
    # Select max and min indexes
    i_max = np.argpartition(tmp_data, -n_max)[-n_max:]
    i_min = np.argpartition(tmp_data, n_min)[:n_min] 
    i_peaks = np.sort(np.append(i_max, i_min))
    # Copy peaks amplitude and time
    for i in range(len(i_peaks)):
        if(i >= NUM_PEAKS):
            break
        # Copy peaks to structure
        input_peaks[n][2*i] = tmp_data[i]
        input_peaks[n][2*i+1] = tmp_time[i] - prev_time
        prev_time = tmp_time[i]

In [20]:
print(input_peaks)

[[1.  1.  2.  1.  1.5 1.  4.  1.  9.  1.  1.  1.  2.3 1.  5.  1. ]
 [1.  1.  2.  1.  1.5 1.  4.  1.  9.  1.  1.  1.  0.  0.  0.  0. ]
 [1.  1.  2.  1.  1.5 1.  4.  1.  9.  1.  1.  1.  2.3 1.  5.  1. ]
 [1.  1.  2.  1.  1.5 1.  4.  1.  9.  1.  1.  1.  2.3 1.  5.  1. ]]
